In [4]:
# Convert GFP data from file 'by_aa.txt' to file 'by_aa_standard.txt':

import csv

aaMutations, std, size, brightness = 1, 2, 3, 4

seq = 'SKGEELFTGVVPILVELDGDVNGHKFSVSGEGEGDATYGKLTLKFICTTGKLPVPWPTLVTTLSYGVQCFSRYPDHMKQHDFFKSAMPEGYVQERTIFFKDDGNYKTRAEVKFEGDTLVNRIELKGIDFKEDGNILGHKLEYNYNSHNVYIMADKQKNGIKVNFKIRHNIEDGSVQLADHYQQNTPIGDGPVLLPDNHYLSTQSALSKDPNEKRDHMVLLEFVTAAGITHGMDELYK'
with open('by_aa_standard.txt', 'w') as f_st:
    with open('sequences.txt', 'w') as f_seq:
        print('mutations', 'fitness', 'std', 'size', 'sequence', file=f_st, sep='\t')
        with open('by_aa.txt') as f:
            csv_reader = csv.reader(f, delimiter='\t')
            rows = f.readlines()
#            row = rows[1]
#            row.strip()
            print(seq, file=f_seq)
            print('wt', '3.7192121319050004', '0.10699178958770761', '3645', seq, file=f_st, sep='\t')
#            print('wt', row[brightness], row[std], row[size], seq, file=f_st, sep='\t')
            for row in rows[2:]:
                row = row[0:-1]
                row = row.split('\t')
                fields = row[aaMutations].split(':')
                mutations = ''
                mut_seq = seq
                flag = False
                for mut in fields:
                    if mutations != '':
                        mutations += ':'
                    mutations += mut[1:]
                    wt_aa, pos, mut_aa = mut[1], int(mut[2:-1]), mut[-1]
                    if mut_aa == '*': # Comment if to consider nonsense mutations
                        flag = True   # Comment if to consider nonsense mutations
                        break         # Comment if to consider nonsense mutations
                    mut_seq = mut_seq[0:pos] + mut_aa + mut_seq[pos + 1:]
                if flag == True:      # Comment if to consider nonsense mutations
                    continue          # Comment if to consider nonsense mutations
                print(mut_seq, file=f_seq)
                print(mutations, row[brightness], row[std], int(float(row[size])), mut_seq, file=f_st, sep='\t')


In [ ]:
# Process equal-fitness genotypes (better run on cluster)

In [ ]:
# Read GFP phenotypes into list
phen = list()
with open('by_aa_standard.txt') as f:
    rows = f.readlines()
    for line in rows[1:]:
        line = line[0:-1]
        fields = line.split('\t')
        phen.append(float(fields[1]))

In [ ]:
# Generate file 'phenDiff.txt' (24Gb), it takes long
with open('phenDiff.txt', 'w') as f:
    for i in range(0, len(phen)):
        for j in range(i + 1, len(phen)):
            print(abs(phen[j] - phen[i]), file=f)

In [ ]:
# Count genotype pairs with statistically indistinguishable fitness

import math

sigma = 0.10699178958770761 * math.sqrt(2)
z0 = 1.96

countSign, countNon = 0, 0
with open('phenDiff.txt') as f:
    for line in f:
        d = float(line[0:-1])
        if d/sigma > 1.96:
            countSign += 1
        else:
            countNon += 1

print(countSign, countNon, countSign + countNon, countSign/(countSign + countNon))

In [ ]:
# Process single mutations

In [3]:
# Read GFP g2p map into dictionary where keys are sequences, and values are fitnesses

gen2phen = dict()
gen2nr = dict()

with open('by_aa_standard.txt') as f:
    rows = f.readlines()
    i = 0
    for line in rows[1:]:
        line = line[0:-1]
        fields = line.split('\t')
        gen2phen[fields[4]] = float(fields[1])
        gen2nr[fields[4]] = i
        i += 1

In [5]:
# Count mutations statistically indistinguishable from neutral

import math

sigma = 0.10699178958770761 * math.sqrt(2)
neutral, non_neutral = 0, 0
with open('neutral.txt', 'w') as out:
    with open('hypercubes/hypercubes_1.txt') as f:
        for line in f:
            line = line[0:-1]
            mut, seq_final = line.split('\t')
            first, pos, second = mut[0], int(mut[1:-1]), mut[-1]
            seq_initial = seq_final[0:pos] + first + seq_final[pos + 1:]
            #print(len(seq_final), len(seq_initial), len(first))
            #print(seq_final, seq_initial, first, pos, sep='\n')

            #f1 = gen2phen[seq_final]
            #f2 = gen2phen[seq_initial]
            if abs(gen2phen[seq_final] - gen2phen[seq_initial]) / sigma <= 1.96: 
                print(mut, 'ns', abs(gen2phen[seq_final] - gen2phen[seq_initial]), file=out)
                neutral += 1
            else:
                print(mut, 's', abs(gen2phen[seq_final] - gen2phen[seq_initial]), file=out)
                non_neutral += 1
                
print(neutral, non_neutral, neutral + non_neutral, neutral/(neutral + non_neutral))

28450 5558 34008 0.8365678663843802
